In [ ]:
# liver_v1

In [ ]:
import numpy as np

In [ ]:
import matplotlib, matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
#! scp intern_repo/new_cell_images/*.npy cell_datasets/liver_v1/seg/

In [ ]:
#! ls intern_repo/new_cell_images/
mask,img=np.load("intern_repo/new_cell_images/21_11_mask.npy"),np.load("intern_repo/new_cell_images/21_11.npy")
# plt.figure()
# plt.imshow()
# plt.figure()
# plt.imshow()




In [ ]:
import brambox as bb
import os
from os.path import join, basename
from pathflowai.utils import load_sql_df, npy2da
import skimage
import dask, dask.array as da, pandas as pd, numpy as np
import argparse
from scipy import ndimage
from scipy.ndimage.measurements import label as scilabel
import pickle
from dask.distributed import Client
from multiprocessing import Pool
from functools import reduce
import histomicstk as htk
import scipy.stats as sts

In [ ]:
# # obj_labels={}
# # for i in range(1,num_classes+1):
# #     obj_labels[i]=np.unique(lbls[m==i].flatten())
# # rev_label={}
# # for k in obj_labels:
# #     for i in obj_labels[k]:
# #         rev_label[i]=k
# # rev_label={k:rev_label[k] for k in sorted(list(rev_label.keys()))}
# objProps = list(skimage.measure.regionprops(lbls))

In [ ]:
import copy,cv2,glob

In [ ]:

cell_images=[]
for mask in glob.glob("intern_repo/new_cell_images/*_mask.npy"):
    img=mask.replace("_mask","")
    base=os.path.basename(img).replace(".npy","")
    print(base)
    mask,img=np.load(mask),np.load(img)
    lbls,n_lbl=scilabel(mask)

    imNucleicompact = htk.segmentation.label.compact(lbls, compaction=3)

    k= (imNucleicompact==-1)
    imNucleicompact1=np.copy(k)
    plt.rcParams['figure.figsize'] = 15,15    
    for ii in range(0,k.shape[0]):
        for jj in range(0,k.shape[1]):
            if imNucleicompact[ii,jj]>0:
                imNucleicompact1[ii,jj]=1

    imNucleicompact2 = skimage.measure.label(imNucleicompact1,connectivity = 1)

    listt = []
    seglis = []
    list_nuclei = []
    right = 0
    segment = 0
    label = 0
    for i in range(1,imNucleicompact2.max()):

        k =  (imNucleicompact2==i)
        location = np.where(k == 1)
        labels = (mask[location]).flatten()
        x_min, y_min = min(location[0]),min(location[1])
        x_max, y_max = max(location[0]),max(location[1])
        space = (x_max-x_min)*(y_max-y_min)

        if space<450 and space>100:           
            segmentate = k[x_min:x_max,y_min:y_max]
            segmentate = np.tile(np.expand_dims(segmentate,axis=2),(1,1,3))
            listt.append([x_min,y_min,x_max,y_max])
            seglis.append(segmentate)
            img1 = copy.deepcopy(img[x_min:x_max,y_min:y_max,:])
            img1 = img1*segmentate
            img1[img1.sum(2)==0]=255
            list_nuclei.append((x_min,x_max,y_min,y_max,sts.mode(labels).mode[0],img1))
            #plt.imshow(img1)
            #plt.show()

        if space>449:
            #print i
            #print space
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7, 7)) 
            k.dtype=np.uint8
            eroded=cv2.erode(k,kernel);
            dilated = cv2.dilate(eroded,kernel)
            new_seg = skimage.measure.label(dilated,connectivity = 1)
            for j in range (1,new_seg.max()+1):
                #print 'j=',j
                kk =  (new_seg==j)
                location1 = np.where(kk == 1)
                labels = (mask[location1]).flatten()
                
                x_min1, y_min1 = min(location1[0]),min(location1[1])
                x_max1, y_max1 = max(location1[0]),max(location1[1])
                space1 = (x_max1-x_min1)*(y_max1-y_min1)
                if space1< 800:
                    segmentate = kk[x_min1:x_max1,y_min1:y_max1]
                    segmentate = np.tile(np.expand_dims(segmentate,axis=2),(1,1,3))
                    listt.append([x_min1,y_min1,x_max1,y_max1])
                    seglis.append(segmentate)
                    img1 = copy.deepcopy(img[x_min1:x_max1,y_min1:y_max1,:])
                    img1 = img1*segmentate
                    img1[img1.sum(2)==0]=255
                    list_nuclei.append((x_min1,x_max1,y_min1,y_max1,sts.mode(labels).mode[0],img1))
    list_nuclei=pd.DataFrame(list_nuclei)
    list_nuclei['basename']=base
    cell_images.append(list_nuclei)
#     break
        

In [ ]:
cell_df=pd.concat(cell_images)
cell_df.columns=['xmin','xmax','ymin','ymax','label','img','basename']

In [ ]:
cell_df.to_pickle("cell_datasets/liver_v1/cluster/liver_nuclei.pkl")

In [ ]:
# skinv1

In [ ]:
import skimage.io

In [ ]:
files=glob.glob("intern_repo/skin_v1/Masks/*")


In [ ]:
plt.imshow((skimage.io.imread(files[0])!=0).sum(2))

In [ ]:
images={}
for f in files:
    mask=f
    base=os.path.splitext(os.path.basename(f))[0]
    img=glob.glob("intern_repo/skin_v1/HE/{}*".format(base))
    if img:
        img=img[0]
        mask=skimage.io.imread(mask).sum(2)!=0
        if mask.flatten().sum():
            lbls=scilabel(mask)[0]
            plt.figure()
            plt.imshow(lbls)
#             print(sts.mode(lbls.flatten()))
            region_props=skimage.measure.regionprops_table(lbls,properties=["centroid"])
#             print(region_props,"HI")
            cell_df=pd.DataFrame(region_props)
            cell_df.columns=['x','y']
            cell_df['basename']=base
            images[base]=dict(img=skimage.io.imread(img),
                             centroids=cell_df)
    
    

In [ ]:
import pickle


In [ ]:
pickle.dump(images,open('cell_datasets/skin_v1/seg/skin_cell_centroids.pkl','wb'))

In [ ]:
# skin_v2 upload second skin 51_

In [ ]:
# ! ln MOHS/images/41_A2c.npy old_data/final_skin/

In [ ]:
! ls old_data/final_skin/ -lah

In [ ]:
import skimage.io
import skimage
import cv2
import tifffile as tiff 
import numpy as np
import tifffile as tiff

# arr=cv2.imread("old_data/final_skin/51_D1b_ASAP.tif")
slide1=True
arr=(tiff.imread("old_data/final_skin/51_D1b_ASAP.tif") if not slide1 else np.load("old_data/final_skin/41_A2c.npy"))


In [ ]:
import xmltodict as xd, numpy as np, pickle
from collections import defaultdict

In [ ]:
with open("old_data/final_skin/{}.xml".format("41_A2c_ASAP" if slide1 else "51_D1b_ASAP"),"rb") as f:
    d=xd.parse(f)


In [ ]:
# d

In [ ]:

cells=[]
cell_dot=[]
for i,annotation in enumerate(d['ASAP_Annotations']["Annotations"]["Annotation"]):
#     print(annotation["@PartOfGroup"])
#     print(annotation)
#     break
#     if annotation["@PartOfGroup"]=="nuc":
    if annotation["@Type"]=="Dot":
        cell_dot.append(np.array((float(annotation["Coordinates"]["Coordinate"]["@X"]),float(annotation["Coordinates"]["Coordinate"]["@Y"]))))
    elif annotation["@Type"]=="Spline":
        try:
            lbl=annotation["@PartOfGroup"]
            contour=np.array([(float(coord["@X"]),float(coord["@Y"])) for coord in annotation["Coordinates"]["Coordinate"]])
            cells.append(contour)
        except:
            print(i)
        

In [ ]:
len(cells)

In [ ]:
import pandas as pd
import matplotlib, matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']=300
import seaborn as sns

In [ ]:
contour_df=pd.DataFrame(cells,columns=['contours'])
contour_df['xmin']=contour_df['contours'].map(lambda x: x[:,0].min())
contour_df['xmax']=contour_df['contours'].map(lambda x: x[:,0].max())
contour_df['ymin']=contour_df['contours'].map(lambda x: x[:,1].min())
contour_df['ymax']=contour_df['contours'].map(lambda x: x[:,1].max())
contour_df['xmean']=contour_df['contours'].map(lambda x: x[:,0].mean())
contour_df['ymean']=contour_df['contours'].map(lambda x: x[:,1].mean())

In [ ]:
dot_annotations=pd.DataFrame(cell_dot,columns=['x','y'])

In [ ]:
contour_df.shape

In [ ]:
plt.scatter(*contour_df[['xmean','ymean']].T.values.tolist())

In [ ]:
plt.scatter(*dot_annotations[['x','y']].T.values.tolist())

In [ ]:
from sklearn.cluster import SpectralClustering, AgglomerativeClustering,Birch,KMeans
from sklearn.mixture import GaussianMixture,BayesianGaussianMixture
from hdbscan import HDBSCAN
sc=KMeans(random_state=42,n_clusters=10) if slide1 else BayesianGaussianMixture(n_components=5,random_state=42,covariance_type="full",init_params="random",weight_concentration_prior_type="dirichlet_distribution")#SpectralClustering(n_clusters=7,affinity='nearest_neighbors',assign_labels="discretize",n_neighbors=7)
c=sc.fit_predict(dot_annotations)#10

In [ ]:
plt.scatter(*dot_annotations.T.values.tolist(),c=c)
len(c),len(contour_df)

In [ ]:
c2=sc.predict(contour_df[['xmean','ymean']])
plt.scatter(*contour_df[['xmean','ymean']].T.values.tolist(),c=c2)

In [ ]:
from sklearn.metrics import silhouette_samples
import seaborn as sns
sns.distplot(silhouette_samples(contour_df[['xmean','ymean']], c2))

In [ ]:
include_contour=(silhouette_samples(contour_df[['xmean','ymean']], c2)>=(0.3 if slide1 else 0.05))
contour_df=contour_df[include_contour]#.mean()#0.5
c2=c2[include_contour]

In [ ]:
dot_annotations['cluster']=c
contour_df['cluster']=c2

In [ ]:
contour_df['perimeter']=contour_df['contours'].map(lambda x: np.sqrt((np.diff(x,axis=0)**2).sum(axis=1)).sum())

In [ ]:
idx_keep=contour_df['perimeter'].values.argsort()[::-1][(4 if slide1 else 8):]
contour_df=contour_df.iloc[idx_keep]
c2=c2[idx_keep]

In [ ]:
plt.scatter(*contour_df[['xmean','ymean']].T.values.tolist(),c=c2)

In [ ]:
images=[]
cluster_box=dict()
for i,(name,dff) in enumerate(dot_annotations.groupby("cluster")):
#     if i==2:
    xmin,xmax,ymin,ymax=dff['x'].min(),dff['x'].max(),dff['y'].min(),dff['y'].max()
    xmin,xmax,ymin,ymax=int(xmin)-20,int(xmax)+20,int(ymin)-20,int(ymax)+20
    cluster_box[name]=[xmin,xmax,ymin,ymax]
    dff.loc[:,'x']=dff['x']-xmin
    dff.loc[:,'y']=dff['y']-ymin
    images.append(dict(dot_annot=dff,
                       region_box=cluster_box[name]))
    plt.figure()
    plt.imshow(arr[ymin:ymax,xmin:xmax,:])
    plt.scatter(dff['x'],dff['y'])


In [ ]:
! ls old_data/

In [ ]:
import numpy as np
import copy
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt

# define pts from the question
def fit_spline(pts):
    try:
        tck, u = splprep(pts.T, u=None, s=0.0, per=1) 
        u_new = np.linspace(u.min(), u.max(), 1000)
        x_new, y_new = splev(u_new, tck, der=0)
        pts=np.vstack((x_new, y_new)).T
    except:
        pass

#     plt.plot(pts[:,0], pts[:,1], 'ro')
#     plt.plot(x_new, y_new, 'b--')
    return pts

for i,(name,dff) in enumerate(contour_df.groupby("cluster")):
#     if i==3:
    xmin,xmax,ymin,ymax=cluster_box[name]
    img=copy.deepcopy(arr[ymin:ymax,xmin:xmax,:])
    images[i]['img']=img
    dff.loc[:,'contours']=dff['contours'].map(lambda x: x-np.array([xmin,ymin])).map(fit_spline)
    images[i]['contours']=dff
    
    plt.figure()
    plt.imshow(arr[ymin:ymax,xmin:xmax,:])
    for contour in dff['contours'].values:
        try:
            plt.plot(*contour.T.tolist())#.T.tolist()
        except:
            pass
#     break

In [ ]:
from skimage import draw
import numpy as np
def poly2mask(vertex_row_coords, vertex_col_coords, shape):
    fill_row_coords, fill_col_coords = draw.polygon(vertex_row_coords, vertex_col_coords, shape)
    mask = np.zeros(shape, dtype=np.bool)
    mask[fill_row_coords, fill_col_coords] = True
    return mask

In [ ]:
contours_df2=contour_df.copy()
contours_df2.loc[:,'contours']=contours_df2['contours'].map(fit_spline)
contours_df2['xmin']=contours_df2['contours'].map(lambda x: x[:,0].min())
contours_df2['xmax']=contours_df2['contours'].map(lambda x: x[:,0].max())
contours_df2['ymin']=contours_df2['contours'].map(lambda x: x[:,1].min())
contours_df2['ymax']=contours_df2['contours'].map(lambda x: x[:,1].max())


In [ ]:
cell_imgs=[]
for i in range(contours_df2.shape[0]):
#     if i==500:
    xmin,xmax,ymin,ymax=contours_df2.iloc[i][['xmin','xmax','ymin','ymax']]
    xmin=int(xmin)
    ymin=int(ymin)
    xmax=int(round(xmax))
    ymax=int(round(ymax))
    contour=contours_df2.iloc[i]['contours']-np.array([xmin,ymin])
    img=copy.deepcopy(arr[ymin:ymax,xmin:xmax,:])
    mask=poly2mask(contour[:,1],contour[:,0],img.shape[:2])
    img[~mask]=255
    cell_imgs.append(img)
#         plt.imshow(img)
#         break

In [ ]:
pickle.dump(images,open("cell_datasets/skin_v2/seg/skin_dot_shape_annotations_images.pkl",'wb'))

In [ ]:
contours_df2['cell_img']=cell_imgs
contours_df2.to_pickle("cell_datasets/skin_v2/cluster/skin_cell_images.pkl")

In [ ]:
# colon

In [ ]:
import numpy as np
arr=np.load("intern_repo/18_G7_3_orig.npy")

In [ ]:
import xmltodict as xd, numpy as np, pickle
from collections import defaultdict

In [ ]:
with open("intern_repo/18_G7_3_orig_nuclei.xml","rb") as f:
    d=xd.parse(f)
#     d_h=None
#     d_h=d['ASAP_Annotations']['AnnotationGroups']


    d_final=defaultdict(list)
#     try:
#         for i,annotation in enumerate(d['ASAP_Annotations']["Annotations"]["Annotation"]):

In [ ]:
cells=[]
cell_dot=[]
for i,annotation in enumerate(d['ASAP_Annotations']["Annotations"]["Annotation"]):
#     print(annotation["@PartOfGroup"])
    if annotation["@PartOfGroup"]=="nuc":
        if annotation["@Type"]=="Dot":
            cell_dot.append(np.array((float(annotation["Coordinates"]["Coordinate"]["@X"]),float(annotation["Coordinates"]["Coordinate"]["@Y"]))))
        elif annotation["@Type"]=="Spline":
            lbl=annotation["@PartOfGroup"]
            contour=np.array([(float(coord["@X"]),float(coord["@Y"])) for coord in annotation["Coordinates"]["Coordinate"]])
            cells.append(contour)
        

In [ ]:
contour_df=pd.DataFrame(cells,columns=['contours'])
contour_df['xmin']=contour_df['contours'].map(lambda x: x[:,0].min())
contour_df['xmax']=contour_df['contours'].map(lambda x: x[:,0].max())
contour_df['ymin']=contour_df['contours'].map(lambda x: x[:,1].min())
contour_df['ymax']=contour_df['contours'].map(lambda x: x[:,1].max())
contour_df['xmean']=contour_df['contours'].map(lambda x: x[:,0].mean())
contour_df['ymean']=contour_df['contours'].map(lambda x: x[:,1].mean())

In [ ]:
plt.scatter(*contour_df[['xmean','ymean']].T.values.tolist())

In [ ]:
dot_annotations=pd.DataFrame(cell_dot,columns=['x','y'])

In [ ]:
from sklearn.cluster import SpectralClustering, AgglomerativeClustering,Birch,KMeans
sc=KMeans(random_state=42,n_clusters=9)#SpectralClustering(n_clusters=7,affinity='nearest_neighbors',assign_labels="discretize",n_neighbors=7)
c=sc.fit_predict(dot_annotations)
c[np.isin(c,[0,6,7])]=0

In [ ]:
c2=sc.predict(contour_df[['xmean','ymean']])
c2[np.isin(c2,[0,6,7])]=0
contour_df['cluster']=c2

In [ ]:
plt.scatter(*dot_annotations.T.values.tolist(),c=c)

In [ ]:
dot_annotations['cluster']=c

In [ ]:
images=[]
cluster_box=dict()
for i,(name,dff) in enumerate(dot_annotations.groupby("cluster")):
#     if i==2:
    xmin,xmax,ymin,ymax=dff['x'].min(),dff['x'].max(),dff['y'].min(),dff['y'].max()
    xmin,xmax,ymin,ymax=int(xmin)-20,int(xmax)+20,int(ymin)-20,int(ymax)+20
    cluster_box[name]=[xmin,xmax,ymin,ymax]
    dff.loc[:,'x']=dff['x']-xmin
    dff.loc[:,'y']=dff['y']-ymin
    images.append(dict(dot_annot=dff,
                       region_box=cluster_box[name]))
    plt.figure()
    plt.imshow(arr[ymin:ymax,xmin:xmax,:])
    plt.scatter(dff['x'],dff['y'])


In [5]:
import numpy as np
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt

# define pts from the question
def fit_spline(pts):
    try:
        tck, u = splprep(pts.T, u=None, s=0.0, per=1) 
        u_new = np.linspace(u.min(), u.max(), 1000)
        x_new, y_new = splev(u_new, tck, der=0)
        pts=np.vstack((x_new, y_new)).T
    except:
        pass

#     plt.plot(pts[:,0], pts[:,1], 'ro')
#     plt.plot(x_new, y_new, 'b--')
    return pts

for i,(name,dff) in enumerate(contour_df.groupby("cluster")):
#     if i==3:
    xmin,xmax,ymin,ymax=cluster_box[name]
    img=copy.deepcopy(arr[ymin:ymax,xmin:xmax,:])
    images[i]['img']=img
    dff.loc[:,'contours']=dff['contours'].map(lambda x: x-np.array([xmin,ymin])).map(fit_spline)
    images[i]['contours']=dff
    
    plt.figure()
    plt.imshow(arr[ymin:ymax,xmin:xmax,:])
    for contour in dff['contours'].values:
        try:
            plt.plot(*contour.T.tolist())#.T.tolist()
        except:
            pass
#     break

NameError: name 'contour_df' is not defined

In [ ]:
pickle.dump(images,open("cell_datasets/colon/seg/colon_dot_shape_annotations_images.pkl",'wb'))

In [ ]:
contour_df

In [ ]:
from skimage import draw
import numpy as np
def poly2mask(vertex_row_coords, vertex_col_coords, shape):
    fill_row_coords, fill_col_coords = draw.polygon(vertex_row_coords, vertex_col_coords, shape)
    mask = np.zeros(shape, dtype=np.bool)
    mask[fill_row_coords, fill_col_coords] = True
    return mask

In [2]:
nuclei=cv2.fillPoly(np.zeros(img.shape[:2]), pts=contour_df['contours_spline'].map(lambda x:x.astype(int)).tolist(), color=1)

NameError: name 'cv2' is not defined

In [ ]:
contours_df2=contour_df.copy()
contours_df2.loc[:,'contours']=contours_df2['contours'].map(fit_spline)
contours_df2['xmin']=contours_df2['contours'].map(lambda x: x[:,0].min())
contours_df2['xmax']=contours_df2['contours'].map(lambda x: x[:,0].max())
contours_df2['ymin']=contours_df2['contours'].map(lambda x: x[:,1].min())
contours_df2['ymax']=contours_df2['contours'].map(lambda x: x[:,1].max())


In [ ]:
cell_imgs=[]
for i in range(contours_df2.shape[0]):
#     if i==500:
    xmin,xmax,ymin,ymax=contours_df2.iloc[i][['xmin','xmax','ymin','ymax']]
    xmin=int(xmin)
    ymin=int(ymin)
    xmax=int(round(xmax))
    ymax=int(round(ymax))
    contour=contours_df2.iloc[i]['contours']-np.array([xmin,ymin])
    img=copy.deepcopy(arr[ymin:ymax,xmin:xmax,:])
    mask=poly2mask(contour[:,1],contour[:,0],img.shape[:2])
    img[~mask]=255
    cell_imgs.append(img)
#         plt.imshow(img)
#         break

In [ ]:
contours_df2['cell_img']=cell_imgs

In [ ]:
contours_df2.to_pickle("cell_datasets/colon/cluster/colon_cell_images.pkl")

In [ ]:
# autoparis

In [ ]:
! ls

In [ ]:
! ls autoparis/images

In [ ]:
cell_dfs=[]
for k in [x.split("_")[1] for x in glob.glob("autoparis/images/*") if x.split("_")[1]!="unsorted"]:
    print(k)
    cell_df=pd.DataFrame(pickle.load(open(f'autoparis/images/all_{k}_arrlist.p','rb')),columns=['image'])
    cell_df['label']=k
    cell_dfs.append(cell_df)

In [ ]:
cell_df=pd.concat(cell_dfs)

In [ ]:
cell_df.shape

In [ ]:
cell_df.to_pickle("cell_datasets/autoparis/cluster/autoparis_cell_images.pkl")

In [ ]:
# tcga

In [ ]:

from numpy import array as to_numpy_array, int32 as npint32
import xml.etree.ElementTree as ET

base="TCGA-E2-A14V-01Z-00-DX1"#"TCGA-49-4488-01Z-00-DX1"#"TCGA-18-5592-01Z-00-DX1"
tcga_imgs=dict()
for f in glob.glob("TCGA_images/*.png"):
    base=f.split("/")[-1].split(".png")[0]
    tree = ET.parse(f"TCGA_images/{base}.xml")

    contours = [
        to_numpy_array(
            [[float(vertex.get("X")), float(vertex.get("Y"))]
                for vertex in region],
            # np.int32 is necessary for cv2.drawContours
            dtype=npint32)
        for region in tree.getroot().iter("Vertices")
    ]
    tcga_imgs[base]=dict(img=skimage.io.imread(f"TCGA_images/{base}.png"),
         contours=contours,
         bboxes=[(contour[:,0].min(),contour[:,0].max(),contour[:,1].min(),contour[:,1].max()) for contour in contours])

In [ ]:
pickle.dump(tcga_imgs,open('cell_datasets/tcga/seg/tcga_segmentation_bbox.pkl','wb'))

In [ ]:
tcga_cell_imgs={}
for k in tcga_imgs:
    tcga_cell_imgs[k]=dict(cell_images=[],
                          bboxes=tcga_imgs[k]["bboxes"])
    for i in range(len(tcga_imgs[k]["bboxes"])):
        xmin,xmax,ymin,ymax=tcga_imgs[k]["bboxes"][i]
        xmin=int(xmin)
        ymin=int(ymin)
        xmax=int(round(xmax))
        ymax=int(round(ymax))
        contour=tcga_imgs[k]['contours'][i]-np.array([xmin,ymin])
        img=copy.deepcopy(tcga_imgs[k]['img'][ymin:ymax,xmin:xmax,:])
        mask=poly2mask(contour[:,1],contour[:,0],img.shape[:2])
        img[~mask]=255
#         plt.figure()
#         plt.imshow(img)
        tcga_cell_imgs[k]['cell_images'].append(img)
    tcga_cell_imgs[k]=pd.DataFrame(tcga_cell_imgs[k])
    tcga_cell_imgs[k]['base']=k
tcga_cell_imgs=pd.concat([tcga_cell_imgs[k] for k in tcga_cell_imgs])

In [ ]:
# plt.imshow(tcga_cell_imgs.iloc[1000,0])

In [ ]:
tcga_cell_imgs.to_pickle("cell_datasets/tcga/cluster/tcga_cells.pkl")

In [ ]:
# matplotlib.rcParams['figure.dpi']=400
# plt.imshow(skimage.io.imread(f"TCGA_images/{base}.png"))
# for contour in contours:
#     plt.plot(*contour.T.tolist(),linewidth=1)

In [ ]:
# tcia

In [ ]:
! ls

In [ ]:
files=glob.glob("tcia_cells/manual_segmentation_data/1197*")[:10]

In [ ]:
imgs=glob.glob("tcia_cells/manual_segmentation_data/*_crop.png")

In [ ]:
# plt.imshow(skimage.io.imread(files[0]))

In [ ]:
files

In [ ]:
! scp tcia_cells/manual_segmentation_data/*_crop.png cell_datasets/tcia/seg/

In [ ]:
cell_dfs=[]
for img in imgs:
    print(img)
    mask=img.replace("_crop","_labeled_mask_corrected")
    img,lbls=skimage.io.imread(img),skimage.io.imread(mask)
    np.save("cell_datasets/tcia/seg/{}".format(os.path.basename(mask).replace(".png",".npy")),lbls)
    
    bboxes=[]
    cells=[]
    lbl_id=sorted(np.unique(lbls.flatten()))
    regions = skimage.measure.regionprops(lbls)
    for i,region in enumerate(regions):
        xmin,ymin,xmax,ymax=region.bbox
        if xmin>0 and ymin>0 and xmax<255 and ymax<255:
            im1=copy.deepcopy(img[xmin:xmax,ymin:ymax,:])
            im1[lbls[xmin:xmax,ymin:ymax]!=lbl_id[i+1]]=255
            cells.append(im1)
            bboxes.append(region.bbox)
    cell_df=pd.DataFrame(list(zip(cells,bboxes)),columns=["cell_image","bbox"])
    cell_df['base']=os.path.basename(mask).split("_labeled_mask_corrected")[0]
    cell_dfs.append(cell_df)

In [ ]:
cell_df=pd.concat(cell_dfs)
cell_df.head()
# bboxes=[]
# cells=[]
# img=skimage.io.imread(files[1])
# lbls=skimage.io.imread(files[2])
# lbl_id=sorted(np.unique(lbls.flatten()))
# regions = skimage.measure.regionprops(lbls)
# for i,region in enumerate(regions):
#     xmin,ymin,xmax,ymax=region.bbox
#     if xmin>0 and ymin>0 and xmax<255 and ymax<255:
#         im1=copy.deepcopy(img[xmin:xmax,ymin:ymax,:])
#         im1[lbls[xmin:xmax,ymin:ymax]!=lbl_id[i+1]]=255
#         plt.figure()
#         plt.imshow(im1)
    
#     print()
# for i in range(1,lbls.max()+1):
#     loc=np.where(lbls==i)
#     xmin

In [ ]:
cell_df.shape

In [ ]:
cell_df.to_pickle("cell_datasets/tcia/cluster/tcia_cell_images.pkl")

In [ ]:
# plt.imshow(skimage.io.imread(files[2]))

In [ ]:
# plt.imshow(skimage.io.imread(files[1]))

In [ ]:
# skin v2 TODO!!!!

In [ ]:
# liver v2

In [ ]:
img=skimage.io.imread("intern_repo/_6151_1_orig.png")

In [ ]:
plt.imshow(img)

In [ ]:
mask=skimage.io.imread("intern_repo/_6151_1_mask_.png")
plt.imshow(mask)

In [ ]:
pd.Series(mask.flatten()).value_counts()

In [ ]:
plt.hist(grayscale.flatten())

In [ ]:
from skimage.color import rgb2gray

grayscale = rgb2gray(mask)

In [ ]:
# plt.imshow()

In [ ]:
cell_mask=(np.abs(grayscale-0.)<0.1) & (np.abs(grayscale-0.)>0.05)

In [ ]:
lbls=scilabel(cell_mask)[0]

In [ ]:
# plt.imshow(np.abs(grayscale-0.8)<0.1)

In [ ]:
# plt.imshow(np.abs(grayscale-0.2)<0.1)

In [ ]:

pd.Series(grayscale.flatten()).value_counts().index.values.tolist()